# Hidden Markov Models and Decoding

Markov chains were just for warm-up. We are really looking for something a bit more complicated. The DNA sequences, namely, depend on the region of the genome. Say, the sequence will have different properties if it would be around the start of the gene, or within the coding region, or in the poly-A tail. When we sequence the genome, all we have is a sequence, and we do not know about regions. We could perhaps infer something about regions from the sequence. In the following, we will consider that the sequence is emitted from the model, and that the emissions depended on the state of the model, the region. The region, that is, the state, is unknown, or hidden. Hence the name, Hidden Markov Models or HMM.

Let us start with example. There is magician around sitting at the street corner, throwing a coin and earning money from people that pass by who are called to guess the results, heads or tails. Somebody told us that this magician cheats (what else?) and uses two coins, where one is a fair coin and the other one is loaded (high probability of heads of 0.95 instead of 0.5 with fair coin). Magician switches the coins only rarely, with probability of 0.05. We are observing its throws, writing down the sequence. It would cool to know which type of the coin he is using and when does he switch.

Here is the graphical depiction of the model:

![Loaded coin model.](figs/07-loaded-coin.jpg)

Some notation first:
* $\pi$ is called a path and is a sequence of hidden states, $\pi_0\pi_1\ldots\pi_L$. An example path is FFLFFLF.
* $x$ is a sequence of emitted symbols. Because this is the sequence we can observe, we call it observed sequence. An example observed sequence from loaded coin model could be oo-o--.
* $a_{kl}$ is a probability of transitions between hidden states $k$ and $l$, where $a_{kl}=P(\pi_i=l|\pi_{i-1}=k)$.
* $a_{0k}$ is a probability of a starting state, taht is, probability that we start at the hidden state $k$.
* $e_k(b)$ is probability of emitting symbol $b$ at hidden state $k$, that is $e_k(b)=P(x_i=b|\pi_i=k)$. Parameters $e_k(b)$ therefore store emission probabilities.

This is not too complicated: we have hidden states and emissions. The transitions between hidden states are defined with simple Markov chain model that uses transition probabilities. Observed sequence is composed of emissions that depend on the hidden state and are specified through emission probabilities defined for each of the hidden states.

Let us present the loaded dice model with some code. We specify the transition probabilities, emission probabilities (given the hidden state), and the start state (magician starts with a fair dice).

In [13]:
T = {"F": {"F": 0.95, "L": 0.05},
     "L": {"F": 0.05, "L": 0.95}}
E = {"F": {"o": 0.5, "-": 0.5},
     "L": {"o": 0.7, "-": 0.3}}
start = "F"

Following is the code that can receive HMM and uses it to generate a sequence.

In [14]:
def generate_hmm_sequence(h, T, E, n):
    """
    HMM sequence given start state,
    transition, emission matrix and sequence length
    """
    s = weighted_choice(E[h])
    yield h, s
    for _ in range(n-1):
        h = weighted_choice(T[h])
        yield h, weighted_choice(E[h])

Let us try this out and generate a sequence of lenght 70:

In [15]:
path, emission = zip(*[i for i in generate_hmm_sequence(start, T, E, 70)])
print("hidden: ", "".join(path))
print("emitted:", "".join(emission))

hidden:  FFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFFFFLLLLLLLLLLLLL
emitted: --oooooo---ooo-oo-ooooo-ooooooo----o-oo--o--o--o-o-o--o-ooooooo-oooo-o


## Joint Probability of $\pi$ and $x$

Given an observed sequence $x$ and a sequence of hidden states, we would like to compute their joint probability. Note this is still academic exercise, and preparation for what is to follow, since in practical applications the hidden states are, well, hidden. Nevertheless, it does not hurt to know their joint probability. Let us just write it down and then think about it:

$$P(x,\pi)=a_{0\pi_1}\prod_{i=1}^L e_{\pi_i}(x_i)a_{\pi_i \pi_{i+1}}$$

We start with the probability of the start symbol, and then traverse the sequence, multiplying the probability of transition of (hidden) states with probability of emitting a symbol at particular state. Simple, yet, again, not very useful as we do not know the hidden sequence.

## Another Example: Dishonest Casino

Here is another example: casino plays with two dices, one fair and one loaded. The loaded one has higher probability of turning 6, while probability for all the sides in the fair dice is the same (as it should be). Here is graphical depiction of the model and its representation in the code.

![Loaded dice model.](figs/07-loaded-dice.jpg)

In [16]:
A = A = ["1", "2", "3", "4", "5", "6"]
E = {"F": {a: 1/6. for a in A},
     "L": {a: 1/10 if a != "6" else 0.5 for a in A}}
T = {"F": {"F": 0.95, "L": 0.05},
     "L": {"F": 0.1, "L": 0.9}}
start = "F"

Let us generate some sequences from this model:

In [17]:
path, emission = zip(*[i for i in generate_hmm_sequence(start, T, E, 70)])
print("hidden: ", "".join(path))
print("emitted:", "".join(emission))

hidden:  FFFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFFFFFFFFFF
emitted: 4135152121655511116562564624613666116645413636636442361514631455423523


## Most Probable State Path: The Viterbi Algorithm

Given is an observed sequence of emitted symbols $x$. Our task is to find the sequence of hidden states $\pi^*$ with the highest joint probability $P(x,\pi^*)$. That is,

$$\pi^*={\rm argmax}_{\pi} P(x,\pi)$$

This problem is solved by a dynamic programming algorithm called after [Andrea Viterbi](https://en.wikipedia.org/wiki/Andrew_Viterbi) (born in 1935 in Bergamo). Suppose that $v_k(i)$ is the probability of the most probable path ending in state $k$ with observation $x_i$. Then, the probability $v_l(i+1)$ is:

$$ v_l(i+1) = e_l(x_{i+1})\max_k(v_k(i) a_{kl}) $$

Notice that $e_l(x_{i+1})$ is the probability of the emitted symbol $x_{i+1}$ in the state $l$. Notice also that this probability is defined since we know the emitted sequence and hence we know about the symbol $x_{i+1}$. To reach the state $l$ at instance $i+1$, we are therefore looking for the path from appropriate previous state $k$ that would maximize the joint probability. But since at instance $i$ the highest joint probability for that state was already computed as $v_k(i)$, probability $v_l(i+1)$ is the highest joint probability at instance $i+1$.

We use Markov property to infer the maximal joint probabilities from the previous states, and previous states only.

What Viterbi does is finding he most probable state path. It decodes the sequence of the hidden states based on the known sequence of observed states. For this reason, the procedure is also referred to as *decoding*.

## Example: Viterbi Algorithm by Hand

Consider the following hidden Markov model for loaded dice model:

![Another loaded dice model.](figs/07-viterbi-dice.jpg)

Notice that we added a special symbol for the transition to start state. We do not know if the magician will start with the fair or the loaded dice, so both states are equally likely start states.

Say that the emitted sequence is:
```
oo--oooo-o
```

Now, let's do Viterbi algorithm. Eventually, this algorithm is dynamic programming algorithm, and hence we have initialization and construction of dynamic programming table. The initialization is this time shown in the first row of the table (the only probable state at the start of the procedure is ${\cal B}$.

$x_i$|${\cal B}$|F|L|
-|-|-|-|
|$1.0$|$0.0$|$0.0$
o||$.5\max(1\times .5,0)=0.25$|$.9\max(1\times .5,0)=.45$
o||$.5\max(.25\times .8,.45\times .1)=.1$|$.9\max(.25\times .2,.45\times .9)=.364$
-||$.5\max(.1\times .8,.364\times .1)=.04$|$.1\max(.1\times .2,.364\times .9)=.032$

Enough, we stop here. Until this point, our hidden sequence was FFF. From here on we should really write the algorithm. I have written one (you'll do it as a part of the homework), and the output I get is:

```
Observed oo--oooo-o
Decode   FFFLLLLLLL
p=7.31e-05
```

So it turns out that magician first uses a fair dice but then switches to the loaded one.

At this stage it is great to do some more programming. Say, we build a program that considers the hidden Markov model, generates the sequence of hidden states and emission, and then tries to decode the hidden states using Viterbi algorithm. I wrote one, and here is an example of one run:

```
Hidden   FFFFFFFFFFFFFFFFFLLLLFFFFFFLLLLLLLFFLLLLLLLLLFFFFFLLLLLLLLLLLLLLLLFLLLLLLLLLLLLLLLLFFFFFFFFFFFFFFLLF
Observed -o--ooo-o--o-o---ooooooo-ooooo-oooooooooooo-oo-o--oooooooooooooooo-oooooooo-ooooooooo-o-oo--o-ooooo-
Decode   FFFFFFFFFFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLLFFFFFFFFFLLLLLFF
p=9.81e-29
```

We can observe that decoded sequence is not exactly equal to the original sequence of hidden states. Why is that? Also, we can note taht the joint probability of the decoded sequence and emissions is rather very very low. Again, why? And how is it possible that our decoded sequence has higher joint probability that joint probability of emissions and original path? In fact, what is the joint probability of the path and emissions? Can you compute one?

## Viterbi Algorithm, This Time More Formally

Initialization:
$v_0(0)=1$, $v_k(0)=0$ for all $k$

Recursion ($i=1\ldots L$):
$$v_l(i)=e_l(x_i)\max_k(v_k(i-1)a_{kl})$$
$${\rm ptr}_i(l)={\rm argmax}_k(v_k(i-1)a_{kl})$$

Termination:
$$P(x, \pi^*)=\max_k(v_k(L)a_{k0})$$
$$\pi^*_L={\rm argmax}_k(v_k(L)a_{k0})$$

Traceback ($i=L\ldots 1$):
$$\pi^*_{i-1}={\rm ptr}_i(\pi^*)$$